In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.layers import Dense, Input, Bidirectional, LSTM, dot, concatenate, Activation
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout, LeakyReLU
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

In [3]:
# class TwoStreamBiLSTM:
#     def __init__(self, hidden_size=1024, no_classes=1):
#         self.hidden_size = hidden_size
#         self.no_classes = no_classes

#     def build(self, input_shape=((15, 512))):
#         input_text =  Input(shape=input_shape)

#         x = Bidirectional(LSTM(self.hidden_size, return_sequences=True, kernel_initializer='glorot_uniform'))(input_text)
#         x = Bidirectional(LSTM(self.hidden_size, return_sequences=True, kernel_initializer='glorot_uniform'))(x)
#         x = Bidirectional(LSTM(self.hidden_size*2, return_sequences=True, kernel_initializer='glorot_uniform'))(x)
# #         x = Bidirectional(LSTM(self.hidden_size*2, return_sequences=True, kernel_initializer='glorot_uniform'))(x)
# #         x = Bidirectional(LSTM(self.hidden_size*2, return_sequences=True, kernel_initializer='glorot_uniform'))(x)

#         x1 = GlobalAveragePooling1D()(x)
#         #x1 = GlobalMaxPooling1D()(x)

#         x = Dense(self.hidden_size*2, kernel_initializer='glorot_uniform')(x1)
#         x = LeakyReLU(0.2)(x)

#         x = Dropout(rate=0.2)(x)

#         output = Dense(self.no_classes, activation='sigmoid', kernel_initializer='glorot_uniform')(x)

#         return Model(inputs=input_text, outputs=output)

In [4]:
class TwoStreamBiLSTM:
    #chng
    def __init__(self, input_shape=(15, 768), hidden_states=256):
        self.input_shape = input_shape
        self.hidden_states = hidden_states
    
    def build(self):
        inp = Input(shape=self.input_shape)
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(inp)
        x1 = LeakyReLU(0.2)(x)
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(x1)
        x = LeakyReLU(0.2)(x)
        
        x = concatenate([x1, x])
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(x)
        x1 = LeakyReLU(0.2)(x)
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(x1)
        x = LeakyReLU(0.2)(x)
        
        x = concatenate([x1, x])
        
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=False, kernel_initializer='glorot_uniform'))(x)
        x = LeakyReLU(0.2)(x)
        
        x = Dense(512, kernel_initializer='glorot_uniform')(x)
        x = LeakyReLU(0.2)(x)
        x = Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid')(x)
        
        model = Model(inputs=inp, outputs=x)
        return model
    def build_bilstm(self):
        inp = Input(shape=self.input_shape)
        
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(inp)
        x = Bidirectional(LSTM(self.hidden_states, return_sequences=True, kernel_initializer='glorot_uniform'))(x)
        
        x1 = GlobalMaxPooling1D(x)
        x2 = GlobalAveragePooling1D(x)
        
        x = concatenate([x1, x2])
        
        x = Dense(512, kernel_initializer='glorot_uniform')(x)
        x = LeakyReLU(0.2)(x)
        x = Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid')(x)
        
        model = Model(inputs=inp, outputs=x)
        return model

In [5]:
import pandas as pd
df = pd.read_csv('../input/bert-embeddings/postwise_comment_embedding_bert.csv')
df

,_unit_id,label,embedding
0,698432761,0,"[[-0.2728136479854584, -0.8870247602462769, -0..."
1,698432762,0,"[[0.18904195725917816, -0.5301364660263062, 0...."
2,698432763,0,"[[0.4969237148761749, -0.2632063627243042, 0.7..."
3,698432764,0,"[[-0.13861827552318573, -0.779614269733429, 0...."
4,698432765,1,"[[-0.3295409381389618, -0.29162853956222534, -..."
...,...,...,...
948,698447713,1,"[[-0.4126046895980835, -0.2747979760169983, 0...."
949,698447782,1,"[[0.38253065943717957, -0.13010767102241516, -..."
950,698447845,0,"[[-0.15763697028160095, -0.7029473185539246, -..."
951,698447928,0,"[[-0.40601351857185364, -0.4909651577472687, 0..."


In [6]:
import numpy as np
def convert_str_to_array(array_str):
    array_str = array_str.replace('[', '')
    array_str = array_str.replace(']', '')
    array_str = array_str.replace(' ', '')
    return np.fromstring(array_str, sep=', ')
for i in range(len(df)):
  arr = df['embedding'][i]
  arr = convert_str_to_array(arr)
  seq = []
  for j in range(0,15):
    #chng
    seq.append(np.asarray(arr[j*768: j*768+768]))
  df['embedding'][i]=np.asarray(seq)
df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,_unit_id,label,embedding
0,698432761,0,"[[-0.2728136479854584, -0.8870247602462769, -0..."
1,698432762,0,"[[0.18904195725917816, -0.5301364660263062, 0...."
2,698432763,0,"[[0.4969237148761749, -0.2632063627243042, 0.7..."
3,698432764,0,"[[-0.13861827552318573, -0.779614269733429, 0...."
4,698432765,1,"[[-0.3295409381389618, -0.29162853956222534, -..."


In [7]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, precision_score, recall_score

kf = KFold(n_splits=4,random_state = 43, shuffle=True)

precision_vals = []
recall_vals = []
f1_vals = []
acc_vals = []
thresholds = []

count = 0

for i in range(1, 400):
  thresholds.append(.002*i+.1)
#print(thresholds)

for train_index, test_index in kf.split(df['embedding'].values):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = df['embedding'].values[train_index], df['embedding'].values[test_index]
  y_train, y_test = df['label'].values[train_index], df['label'].values[test_index]
  print(len(X_train))
  print(len(X_test))

  monitor = 'val_recall_at_precision'
  if(count!=0):
    monitor += ('_'+str(count))
  #print(monitor)
  count+=1

  callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor= monitor, patience=10, min_delta=0, restore_best_weights=True, mode='max'),
    #tf.keras.callbacks.LearningRateScheduler(warmup, verbose=0),
    #tf.keras.callbacks.ReduceLROnPlateau(monitor=monitor, patience=2, mode='auto', min_delta=0.001, cooldown=0, min_lr=1e-6)
]
  model = TwoStreamBiLSTM()
  model = model.build()
  model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy', tf.keras.metrics.RecallAtPrecision(.70)])
  # model.summary()

  X = []
  y = []
  X_val = []
  y_val = []

  for i in range(len(X_train)):
    try:
      X.append(np.array(X_train[i],dtype=np.float32))
      y.append(np.array([y_train[i]],dtype=np.float32))
    except:
      pass

  for i in range(len(X_test)):
    try:
      X_val.append(np.array(X_test[i],dtype=np.float32))
      y_val.append(np.array([y_test[i]],dtype=np.float32))
    except:
      pass

#   print(len(X))
#   print(len(y))
  model.fit(np.array(X), np.array(y), epochs=50, validation_data = (np.array(X_val), np.array(y_val)), callbacks = callbacks)

  X = []
  y = []
  for i in range(len(X_test)):
    try:
      X.append(np.array(X_test[i],dtype=np.float32))
      y.append(np.array([y_test[i]],dtype=np.float32))
    except:
      pass
  print(len(X))
  print(len(y))
  y_predicted = model.predict(np.array(X))
  print(y_predicted[:10])


  pvals = []
  rvals = []
  fvals = [] 
  avals = []

  for threshold in thresholds:
    y_pred = np.copy(y_predicted)
    #print(y_pred[:10])
    for i in range(len(y_pred)):
      if y_pred[i][0]>=threshold:
        y_pred[i][0]=1
      else:
        y_pred[i][0]=0
    #print(y_pred[:10])

    #matrix = confusion_matrix(y,y_pred)
    #print(matrix)
    #pre = precision_score(y,y_pred)
    #print("Precision: ",pre)
    #pvals.append(pre)
    #rec = recall_score(y,y_pred)
    #print("Recall: ",rec)
    #rvals.append(rec)
    f1 = f1_score(y, y_pred)
    if(f1>.55):
        print(round(threshold,3), ": ",f1, end=' ')
    fvals.append(f1)
    acc = accuracy_score(y, y_pred)
    #print("Accuracy: ",acc)
    avals.append(acc)
  #precision_vals.append(pvals)
  #recall_vals.append(rvals)
  fvals.sort(reverse=True)
  print('max_f1 = ', fvals[0])
  f1_vals.append(fvals)
  acc_vals.append(avals)

#print(f1_vals)

714
239
Epoch 1/50
23/23 [==============================] - 21s 225ms/step - loss: 0.6090 - accuracy: 0.6543 - recall_at_precision: 0.0196 - val_loss: 0.5134 - val_accuracy: 0.7615 - val_recall_at_precision: 0.4085
Epoch 2/50
23/23 [==============================] - 1s 35ms/step - loss: 0.5308 - accuracy: 0.7698 - recall_at_precision: 0.3464 - val_loss: 0.5249 - val_accuracy: 0.7573 - val_recall_at_precision: 0.4507
Epoch 3/50
23/23 [==============================] - 1s 35ms/step - loss: 0.4915 - accuracy: 0.7553 - recall_at_precision: 0.4664 - val_loss: 0.5040 - val_accuracy: 0.7824 - val_recall_at_precision: 0.4507
Epoch 4/50
23/23 [==============================] - 1s 35ms/step - loss: 0.4569 - accuracy: 0.8232 - recall_at_precision: 0.7289 - val_loss: 0.5013 - val_accuracy: 0.7782 - val_recall_at_precision: 0.4930
Epoch 5/50
23/23 [==============================] - 1s 36ms/step - loss: 0.4752 - accuracy: 0.7903 - recall_at_precision: 0.6637 - val_loss: 0.5316 - val_accuracy: 0.7782

In [8]:
f1 = []
acc = {}
rec = {}
prec = {}

for i in range(len(f1_vals[0])):
    f1_avg = (f1_vals[0][i]+f1_vals[1][i]+f1_vals[2][i]+f1_vals[3][i])/4
    if(f1_avg>.55):
        f1.append((f1_avg, thresholds[i]))
#   acc[thresholds[i]]=(acc_vals[0][i]+acc_vals[1][i]+acc_vals[2][i]+acc_vals[3][i])/4
#   rec[thresholds[i]]=(recall_vals[0][i]+recall_vals[1][i]+recall_vals[2][i]+recall_vals[3][i])/4
#   prec[thresholds[i]]=(precision_vals[0][i]+precision_vals[1][i]+precision_vals[2][i]+precision_vals[3][i])/4
f1.sort(reverse=True)
for score, threshold in f1:
    print(round(threshold, 3), ": ", score)
# print(acc)
# print(rec)
# print(prec)


0.106 :  0.6778910059645413
0.104 :  0.6778910059645413
0.102 :  0.6778910059645413
0.108 :  0.6761844066785927
0.11 :  0.675237436981623
0.112 :  0.6744683620237164
0.116 :  0.6741915070956987
0.114 :  0.6741915070956987
0.118 :  0.6730945763496985
0.122 :  0.6719962609323892
0.12 :  0.6719962609323892
0.126 :  0.6709443675242546
0.124 :  0.6709443675242546
0.128 :  0.6699828290627161
0.132 :  0.6693091337157155
0.13 :  0.6693091337157155
0.134 :  0.6685364185545113
0.136 :  0.6681925759815108
0.144 :  0.6679298329442012
0.142 :  0.6679298329442012
0.14 :  0.6679298329442012
0.138 :  0.6679298329442012
0.146 :  0.6676010428915948
0.148 :  0.6668304985124037
0.15 :  0.6661700584392372
0.154 :  0.6661534316768003
0.152 :  0.6661534316768003
0.156 :  0.6652237327410089
0.158 :  0.6651162243553548
0.16 :  0.6646478610667768
0.162 :  0.6636879254431756
0.164 :  0.6624412787145555
0.166 :  0.6620949383774508
0.17 :  0.6609780858581656
0.168 :  0.6609780858581656
0.172 :  0.6608897778397976
